In [174]:
from dataclasses import dataclass, field
from typing import NewType, Union, Literal, Set
import string

Domains 
The relational model uses several domains specific to the scenario, which we define below. 

Let Nn = {i i ∈ N∧i ≤ n}betheset of natural numbers less than or equal to n. 

Let Char be the set of characters defined by the ISO basic Latin Alphabet, with Lower and Upper representing the minuscule and majuscule character subsets, respectively. 

Let String be the set of words over Char extended with spaces.

Character Set

In [ ]:
Lower: Set[str] = set(chr(c) for c in range(ord('a'), ord('z') + 1))
Upper: Set[str] = set(chr(c) for c in range(ord('A'), ord('Z') + 1))
Char = NewType('Char', str)  


In [ ]:

String = NewType('String', str)


• An Hour is represented by two digits using the 24-hour clock, e.g. 04 for 4am and 21 for 9pm. 

Hour = N24

Hour∞ extends Hour with a notion of infinity, 

denoted ω. Hour∞ = Hour∪{ω}

In [177]:
Hour    = NewType('Hour', int)
HourInf = Union[Hour, Literal['ω']]
def is_valid_hour(h: int) -> bool:
    return 0 <= h <= 23

AMinute is similarly represented by two digits. 

Minute = N59 Minute∞ 

extends Minute with a notion of infinity, denoted ω. Minute∞ = Minute∪{ω}

In [ ]:

Minute    = NewType('Minute', int)
MinuteInf = Union[Minute, Literal['ω']]
def is_valid_minute(m: int) -> bool:
    return 0 <= m <= 59

• A Time is represented the concatenation of Hour and Minute,

 e.g. 1345 for 1:45pm. Time = {hm h∈Hour∧m∈Minute}

In [179]:
Time = NewType('Time', str)
def is_valid_time(t: str) -> bool:
    if len(t) != 4 or not t.isdigit():
        return False
    hh, mm = int(t[:2]), int(t[2:])
    return is_valid_hour(hh) and is_valid_minute(mm)

A Stn Code is a three-character string of majuscule characters. StnCode = {abc a,b,c ∈ Upper}

In [180]:
StnCode = NewType('StnCode', str)
def is_valid_stncode(code: str) -> bool:
    return len(code) == 3 and all(ch in Upper for ch in code)

• A TrainID is a six-digit natural number. 

TrainID = {abcdef a,b,c,d,e,f ∈ N9}

In [181]:
TrainID = NewType('TrainID', str)
def is_valid_trainid(tid: str) -> bool:
    return len(tid) == 6 and tid.isdigit()

A Headcode is a four-character string, comprising train class, destination area, and an identifier.

Headcode = {cdij c,i,j ∈ N9 ∧d ∈ Upper}

In [182]:
Headcode = NewType('Headcode', str)
def is_valid_headcode(hc: str) -> bool:
    return (
        len(hc) == 4 and
        hc[0].isdigit() and
        hc[1] in Upper and
        hc[2].isdigit() and
        hc[3].isdigit()
    )

• Company: code for a subset of current train operating companies. 

Company = {VT,CS,XC,SR,GR,GW}

In [183]:
Company = Literal['VT', 'CS', 'XC', 'SR', 'GR', 'GW']

Locations and Stations 

• location(loc : String) 

• station(loc : Πloc(location),code : StnCode) 

Locations are identified by their names. Some locations are stations, which have Computer Reservation System (CRS) codes.

In [ ]:
String   = NewType('String', str)
StnCode  = NewType('StnCode', str)

def is_valid_stncode(code: str) -> bool:
    return len(code) == 3 and code.isupper()

@dataclass(frozen=True)
class Location:
    loc: String

    def __post_init__(self):
        if not isinstance(self.loc, str) or not self.loc:
            raise ValueError(f"invalid location name {self.loc}")

@dataclass(frozen=True)
class Station:
    loc: String             
    code: StnCode

    def __post_init__(self):
        if not is_valid_stncode(self.code):
            raise ValueError(f"invalid CRS code: {self.code}")
 


Trains 

• train(uid : TrainID) 

• coach(uid : Πuid(train),lid : Upper) 

A(passenger) train is a unit formed of a locomotive and a set of coaches, and is identified by its Unit ID. Each coach has a letter designation, which is unique in the context of the train.

In [ ]:
from dataclasses import dataclass, field
from typing import NewType, ClassVar, Set, Dict

# Domain definitions
TrainID = NewType('TrainID', str)
UpperChar = NewType('UpperChar', str)

def is_valid_trainid(tid: str) -> bool:
    return len(tid) == 6 and tid.isdigit()

def is_valid_upper_char(ch: str) -> bool:
    return len(ch) == 1 and 'A' <= ch <= 'Z'

@dataclass
class Train:

    """

    represents a train unit, identified by a unique six-digit TrainID


    """
    uid: TrainID



    _registry: ClassVar[Set[str]] = set()

    def __post_init__(self):
        if not is_valid_trainid(self.uid):
            raise ValueError(f"Invalid TrainID: {self.uid}")
        if self.uid in Train._registry:
            raise ValueError(f"TrainID '{self.uid}' already exists")
        Train._registry.add(self.uid)

@dataclass
class Coach:
    """
    represents a coach attached to a train.
    - uid: references an existing Train.uid
    - lid: single uppercase letter unique within that train
    """
    uid: TrainID
    lid: UpperChar

    _coach_map: ClassVar[Dict[str, Set[str]]] = {}

    def __post_init__(self):

        if self.uid not in Train._registry:
            raise ValueError(f"TrainID '{self.uid}' not defined, create the Train first")


        if not is_valid_upper_char(self.lid):
            raise ValueError(f"invalid coach letter: {self.lid}")


        coaches = Coach._coach_map.setdefault(self.uid, set())
        if self.lid in coaches:
            raise ValueError(f"coach letter '{self.lid}' already used on train '{self.uid}'")
        coaches.add(self.lid)


if __name__ == "__main__":
    t1 = Train(uid="123456")
    cA = Coach(uid="123456", lid="A")
    cB = Coach(uid="123456", lid="B")
    print(t1, cA, cB)

    try:
        Coach(uid="123456", lid="A")
    except ValueError as e:
        print("Error:", e)


Train(uid='123456') Coach(uid='123456', lid='A') Coach(uid='123456', lid='B')
Error: coach letter 'A' already used on train '123456'


Services 

• service(hc : Πhc(route),dh : Hour,dm : Minute,pl : N,uid : Πuid(train),toc : Company) 

• route(hc : Headcode,orig : Πloc(station)) 

• plan(hc : Πhc(route),frm : Πloc(location),loc : Πloc(location),ddh : Hour∞,ddm : Minute∞) 

• stop(hc : Πhc(plan),frm : Πfrm(plan),loc : Πloc(plan),adh : Hour,adm : Minute,pl : N)


In [ ]:
@dataclass(frozen=True)
class Services:
    """
    Represents a train service:
    - hc: headcode (links to Route)
    - dh, dm: departure time at origin (hour, minute)
    - pl: platform at origin
    - uid: TrainID
    - toc: operating Company
    """
    hc: Headcode
    dh: Hour
    dm: Minute
    pl: int
    uid: TrainID
    toc: Company

    def __post_init__(self):
        if not is_valid_hour(self.dh):
            raise ValueError(f"invalid departure hour: {self.dh}")
        if not is_valid_minute(self.dm):
            raise ValueError(f"invalid departure minute: {self.dm}")
        if self.pl < 0:
            raise ValueError(f"platform must be non-negative: {self.pl}")
        if not is_valid_trainid(self.uid):
            raise ValueError(f"invalid TrainID: {self.uid}")
        if self.toc not in ('VT', 'CS', 'XC', 'SR', 'GR', 'GW'):
            raise ValueError(f"invalid company code: {self.toc}")


if __name__ == "__main__":
    svc = Services(hc="1L27", dh=18, dm=59, pl=1, uid="170406", toc="VT")
    print(svc)


Services(hc='1L27', dh=18, dm=59, pl=1, uid='170406', toc='VT')


In [ ]:
from dataclasses import dataclass
from typing import NewType, Union, Literal, ClassVar, Set

@dataclass(frozen=True)
class Route:

    """

    defines a service route
    - hc: headcode (identifies the route)
    - orig: origin station code

    """
    hc: Headcode
    orig: str  

    def __post_init__(self):
        if not is_valid_headcode(self.hc):
            raise ValueError(f"invalid headcode: {self.hc}")
        if not (isinstance(self.orig, str) and len(self.orig)==3 and self.orig.isupper()):
            raise ValueError(f"invalid station code: {self.orig}")

@dataclass(frozen=True)
class Plan:

    hc: Headcode
    frm: str
    loc: str
    ddh: HourInf
    ddm: MinuteInf

    def __post_init__(self):
        if not is_valid_headcode(self.hc):
            raise ValueError(f"invalid headcode: {self.hc}")
        if self.ddh != 'ω' and not is_valid_hour(self.ddh):
            raise ValueError(f"invalid departure differential hours: {self.ddh}")
        if self.ddm != 'ω' and not is_valid_minute(self.ddm):
            raise ValueError(f"invalid departure differential minutes: {self.ddm}")

@dataclass(frozen=True)
class Stop:

    hc: Headcode
    frm: str
    loc: str
    adh: Hour
    adm: Minute
    pl: int

    def __post_init__(self):
        if not is_valid_headcode(self.hc):
            raise ValueError(f"invalid headcode: {self.hc}")
        if not is_valid_hour(self.adh):
            raise ValueError(f"invalid arrival differential hours: {self.adh}")
        if not is_valid_minute(self.adm):
            raise ValueError(f"invalid arrival differential minutes: {self.adm}")
        if not isinstance(self.pl, int) or self.pl < 0:
            raise ValueError(f"platform must be a non-negative integer: {self.pl}")

@dataclass(frozen=True)
class Service:


    hc: Headcode
    dh: Hour
    dm: Minute
    pl: int
    uid: str  # TrainID assumed valid six-digit string
    toc: Company

    def __post_init__(self):
        if not is_valid_headcode(self.hc):
            raise ValueError(f"invalid headcode {self.hc}")
        if not is_valid_hour(self.dh):
            raise ValueError(f"invalid departure hour {self.dh}")
        if not is_valid_minute(self.dm):
            raise ValueError(f"invalid departure minute {self.dm}")
        if not isinstance(self.pl, int) or self.pl < 0:
            raise ValueError(f"platform must be a non-negative integer {self.pl}")
        if not (isinstance(self.uid, str) and len(self.uid) == 6 and self.uid.isdigit()):
            raise ValueError(f"invalid TrainID {self.uid}")
        if self.toc not in ('VT', 'CS', 'XC', 'SR', 'GR', 'GW'):
            raise ValueError(f"invalid company code: {self.toc}")

#test with example route
if __name__ == "__main__":
    r = Route(hc="1L27", orig="EDB")
    p = Plan(hc="1L27", frm="edinburgh", loc="haymarket", ddh=0, ddm=5)
    s = Stop(hc="1L27", frm="edinburgh", loc="haymarket", adh=0, adm=4, pl=2)
    svc = Service(hc="1L27", dh=18, dm=59, pl=1, uid="170406", toc="VT")
    print(r, p, s, svc)


Route(hc='1L27', orig='EDB') Plan(hc='1L27', frm='edinburgh', loc='haymarket', ddh=0, ddm=5) Stop(hc='1L27', frm='edinburgh', loc='haymarket', adh=0, adm=4, pl=2) Service(hc='1L27', dh=18, dm=59, pl=1, uid='170406', toc='VT')


In [188]:
if __name__ == "__main__":
    # Valid examples
    print(is_valid_hour(4))         # True
    print(is_valid_time("1345"))    # True
    print(is_valid_stncode("EDB"))  # True
    print(is_valid_headcode("1L27"))# True

    # Invalid examples
    print(is_valid_hour(24))        # False
    print(is_valid_time("2360"))    # False
    print(is_valid_stncode("edb"))  # False
    print(is_valid_headcode("AB12"))# False

    #custom error messages , 

True
True
True
True
False
False
False
False


Query One: 



In [ ]:
from dataclasses import dataclass
from typing import NewType, List, Literal

@dataclass(frozen=True)
class TrainLEV:
    hc: Headcode
    orig: str
    dep: str  

def trainLEV(services: List[Service], routes: List[Route], train_id: str) -> List[TrainLEV]:
    
    origin_map = {r.hc: r.orig for r in routes}

    filtered = [s for s in services if s.uid == train_id]


    records = []
    for s in filtered:
        hh = f"{s.dh:02d}"
        mm = f"{s.dm:02d}"
        dep_time = hh + mm
        orig = origin_map.get(s.hc, "UNK")
        records.append(TrainLEV(hc=s.hc, orig=orig, dep=dep_time))

    records.sort(key=lambda rec: (int(rec.dep[:2]), int(rec.dep[2:])))
    return records

if __name__ == "__main__":
    routes = [
        Route(hc="1L27", orig="EDB"),
        Route(hc="2S45", orig="GLA"),
    ]
    services = [
        Service(hc="1L27", dh=18, dm=59, pl=1, uid="170406", toc="VT"),
        Service(hc="2S45", dh=9,  dm=5,  pl=2, uid="170406", toc="CS"),
        Service(hc="3A12", dh=14, dm=30, pl=1, uid="999999", toc="XC"),
    ]
    result = trainLEV(services, routes, "170406")
    for r in result:
        print(r)


TrainLEV(hc='2S45', orig='GLA', dep='0905')
TrainLEV(hc='1L27', orig='EDB', dep='1859')


Query 2:

Produce a table that lists all services that depart Edinburgh (EDB).

 The query should return one row per service, and should conform to the following schema. 
 
 scheduleEDB(hc : Headcode,dep : Time,pl : Nat,dest : Stn,len : Nat,toc : Company) 
 
 Here, hc is the service’s headcode and dep is the departure time from its origin station. 
 
 pl is the platform from which the service will depart Edinburgh, len is the number of coaches the service’s train has, and toc is the train operating company. dest should be the next planned location for the service. 
 
 Alternatively, dest will preferably be the destination station for the service. Services should be listed in ascending order of departure time.

In [ ]:
@dataclass(frozen=True)
class ScheduleEDB:
    hc: Headcode
    dep: Time        
    pl: int           
    dest: StnCode     
    length: int      
    toc: Company      

def scheduleEDB(
    services: List[Service],
    routes:   List[Route],
    plans:    List[Plan],
    stations: List[Station],
    coaches:  List[Coach]

) -> List[ScheduleEDB]:
    


    orig_map = {r.hc: r.orig for r in routes}



    code_to_loc = {s.code: s.loc for s in stations}
    loc_to_code = {s.loc: s.code for s in stations}


   
    coach_counts = {}
    for c in coaches:
        coach_counts[c.uid] = coach_counts.get(c.uid, 0) + 1

    results: List[ScheduleEDB] = []





    edb_loc = code_to_loc.get("EDB")

    for svc in services:




        if orig_map.get(svc.hc) != "EDB":
            continue




        dep_str = f"{svc.dh:02d}{svc.dm:02d}"
        dep_time = Time(dep_str)

        
        next_plan = next(
            (p for p in plans if p.hc == svc.hc and p.frm == edb_loc),
            None
        )
        if next_plan and next_plan.loc in loc_to_code:
            dest_code = loc_to_code[next_plan.loc]
        else:




            final_plan = next(
                (p for p in plans if p.hc == svc.hc and p.ddh == 'ω'),
                None
            )
            dest_code = loc_to_code.get(final_plan.loc) if final_plan else "UNK"

        length = coach_counts.get(svc.uid, 0)

        results.append(ScheduleEDB(
            hc=svc.hc,
            dep=dep_time,
            pl=svc.pl,
            dest=StnCode(dest_code),
            length=length,
            toc=svc.toc
        ))

    results.sort(key=lambda r: (int(r.dep[:2]), int(r.dep[2:])))
    return results

ScheduleEDB(hc='2S45', dep='0905', pl=2, dest='GLA', length=8, toc='CS')
ScheduleEDB(hc='1L27', dep='1859', pl=1, dest='HYM', length=8, toc='VT')

#this doesnt work properly yet, need to check the logic in the function

ScheduleEDB(hc='1L27', dep='1859', pl=1, dest='HYM', length=8, toc='VT')

Query 3:

Produce a table that lists all the locations through which the 1859 1L27 service departs, and includes both the service’s origin and destination locations. 

The query should return one row per location, and should conform to the following schema. 

serviceEDBDEE(loc : Loc,stn : StnCode,pl : Nat,arr : Time,dep : Time) 

Here, loc is the origin, destination, or a location from which the service departs. 

Should loc be a station, stn gives the corresponding station code, and pl gives the corresponding platform.

Arrival and departure times from that location are recorded under arr and dep, respectively. 
 
Locations should be listed in the order that they are visited.

Constraints Enforce each of the following constraints.

 You may use checks, triggers, procedures, or any other appropriate mechanism. 
 
 1. A service cannot arrive after it departs a station.  

2. All locations on a route, barring the destination location, must have a finite departure differential.

3. A train cannot be part of two different services departing at the same time.